# ***Solving PDEs with Gridap***

In this tutorial we will go through the fundamental theory of the Finite Element Method to solve PDEs. This theory expects little to no knowledge in FEM and is written as a roadmap which can be followed to solve most PDEs using Gridap. 

To apply the theory we will solve the Poisson equation. After each piece of theory, we apply the theory to the problem to demonstrate the workflow presented in the tutorial.

Let's begin!

## **What is a (Partial) Differential Equation P(DE)?**

To describe phenomena in the real world we need to understand the physics behind the phenomena. Physics can often be described in words, but most often we want to describe the physics using mathematical equations. For these kind of problems we always have two kind of variables: 

1. Independent variables: such as space ($\mathbf{x}$) and time ($t$).
2. Dependent variables: a variable that depends on the independent variables (for instance, the deformation of a beam along its span (thus space) and in time)

The relation between independent and dependent variables is given by a (partial) differential equation, along with initial and boundary conditions such that we can solve the (P)DE and find the ***dependent*** variables, which we initially do not know. Hence, a P(DE) only describes the relation between the dependent and independent variables, from which we can then *find* the unknown dependent variables.

In this theory we will mainly focus on **Partial Differential Equations** since these are regarded more difficult to solve using analytical approaches,in comparison to Ordinary Differential Equations. 

One of the most known PDEs is the ***Poisson Equation***. Hence, in this tutorial we will solve the ***Poisson Equation*** to demonstrate the purposed roadmap. Keep in mind that the purposed roadmap can be (in most cases) applicable for other PDEs.

# Solve Poisson equation on a 2D square

Basic steps for FEM:
1. Find the physical problem
2. Describe the mathematical model
    1. Governing equations (PDEs, boundary conditions, initial conditions)
    2. Constant parameters , source functions
    3. Define the unknown quantities to solve for
3. Create FE space to find the unknown quantities in.
    1. Create mesh
        1. Mesh density & mesh profile
        2. Type of FE spaces
        3. Conformity of the FE functions 
4. Writing the weak form of the problem.
5. Solving the problem.
6. Iterating for different mesh sizes until solution converges.

# **Step 1: Define physical problem**

The very first step for all problems is to define the physical problem. This step is for the largest part unconnected to the Finite Element Method but is essential in order to be able to continue. For this we need to understand some terminology and symbols which we often encounter in PDEs.
## **Scalar and vector valued variables**
For simplicity, we consider *two* type of variables: ***scalar valued variables*** and ***vector valued variables***.

### ***Scalar valued variables***
A ***scalar valued variable*** is a variable which, as the name says, is a scalar. Imagine we would like to know the deformation of a beam in space and time. Hence, we want to find a dependent variable, in this case the deformation of the beam denoted as $u$, as a function of the space and time (the independent variables). For a 1D beam we can describe the deformation as:
$$
    \text{deformation of the beam at } x_{0} \text{ and } t_{0} : u(x_{0},t_{0}) = \text{a scalar which describes the deformation at } x_{0} \text{ and } t_{0}
$$
Mathematically we denote this as:
$$
u : \mathbb{R}^d \times \mathbb{R} \rightarrow \mathbb{R}
$$
which means that the deformation can be explained be a function $u$ that has two inputs: the space of the beam, which has $d$ dimensions (for a 1D beam, $d$=1 etc.), and the second input is time (which is always "1D"). Note that for the deformation of a plate, the space would not be 1-dimensional anymore, since we then would consider $\mathbf{x} = (x,y)$. In that case our space is a *vector-valued variable*, but our dependent variable, $u$, remains a scalar-valued variable since the deformation of a plate is still a scalar along the space of the plate.

### ***Vector valued variables***
A ***vector valued variable*** is a variable which, as the name says, is a vector. Vector valued variables describe a quantity using a vector, and not as a scalar. One example is the velocity of a fluid. Imagine we want to describe the velocity of a fluid in a 2D space. The velocity at any point in space and time can be described by a vector with two components (one for each spatial dimension). Mathematically, we denote this as:

$$
\mathbf{v} : \mathbb{R}^d \times \mathbb{R} \rightarrow \mathbb{R}^d
$$

which means that the velocity can be explained by a function $\mathbf{v}$ that has two inputs: the space of the fluid, which has $d$ dimensions (for a 2D space, $d$=2), and the second input is time. The output of this function is a vector with $d$ components, each representing the velocity in one of the spatial dimensions. For example, in a 2D space, the velocity at a point $(x_0, y_0)$ and time $t_0$ can be written as:

$$
\mathbf{v}((x_0, y_0), t_0) = \begin{pmatrix} v_x((x_0, y_0), t_0) \\ v_y((x_0, y_0), t_0) \end{pmatrix}
$$

where $v_x$ and $v_y$ are the velocity components in the $x$ and $y$ directions, respectively.

## **Symbols**

The Finite Element Method relies on a strong understanding of mathematics and is often formulated using mathematical symbols which *set* the "rules" and describe the relation between the dependent and dependent variables. 

### **Operators**

In the context of PDEs and FEM, operators are mathematical tools that help us describe the relationships between variables. Here are some common operators:

1. **Gradient ($\nabla$)**: The gradient operator takes a scalar field and produces a vector field. It represents the rate and direction of change in the scalar field. For a function $u(x, y)$, the gradient is:
    $$
    \nabla u = \begin{pmatrix} \frac{\partial u}{\partial x} \\ \frac{\partial u}{\partial y} \end{pmatrix}
    $$

2. **Divergence ($\nabla \cdot$)**: The divergence operator takes a vector field and produces a scalar field. It measures the magnitude of a source or sink at a given point. For a vector field $\mathbf{v} = (v_x, v_y)$, the divergence is:
    $$
    \nabla \cdot \mathbf{v} = \frac{\partial v_x}{\partial x} + \frac{\partial v_y}{\partial y}
    $$

3. **Laplacian ($\Delta$)**: The Laplacian operator is the divergence of the gradient of a scalar field. It is often used in diffusion problems. For a function $u(x, y)$, the Laplacian is:
    $$
    \Delta u = \nabla \cdot (\nabla u) = \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}
    $$

### **Boundary Conditions**
Boundary and initial conditions are essential for solving Partial Differential Equations (PDEs) because they provide the necessary information to obtain a ***unique*** solution. 

Boundary conditions are, as the name says, used to assign the value of the unknown dependent variable at the boundaries of the considered domain. Boundary conditions are essential to solve the problem in *space*.

Boundary conditions specify the behavior of the solution at the boundaries of the domain. There are typically two types of boundary conditions:
1. **Dirichlet Boundary Condition**: Specifies the value of the solution at the boundary. For example, if we are solving for temperature distribution, a Dirichlet boundary condition might specify the temperature at the boundary.
    $$
    u = g \quad \text{on } \Gamma_D
    $$

2. **Neumann Boundary Condition**: Specifies the value of the derivative (flux) of the solution normal to the boundary. For example, in heat transfer problems, a Neumann boundary condition might specify the heat flux at the boundary.
    $$
    \nabla u \cdot \mathbf{n} \equiv \frac{\partial u}{\partial n} =  h \quad \text{on } \Gamma_N
    $$

Often it can be difficult to understand the definition of a "boundary", especially for beginners. However, a **boundary** of a **domain** "lives" in the *(n-1)*-dimension of a domain that is defined in *n*-dimensions. 

That is, let $\Omega \subset \mathbb{R}^n$ be an open domain where the governing PDE is defined. The $\textbf{boundary}$ of the domain, denoted as $\partial \Omega$, is the set of points that separate the interior of $\Omega$ from its exterior.

Mathematically:
$$
\partial \Omega = \overline{\Omega} \setminus \Omega,
$$
where:
1. $\overline{\Omega}$: The $\textbf{closure}$ of $\Omega$, which includes all points in $\Omega$ and all its boundary points.
2. $\Omega$: The $\textbf{interior}$ of the domain, excluding the boundary. 
3. $\setminus$: Denotes the set difference. 
    
In simpler terms, $\partial \Omega$ consists of all points $\mathbf{x} \in \mathbb{R}^n$ that act like a barrier between the domain $\Omega$ and its complement $\mathbb{R}^n \setminus \Omega$ (the "outside world").


<div style="text-align: center;">
  <img src="images/boundaries.png" alt="Domain Poisson" width="600"/>
</div>

### **Determining the Number of Boundary Conditions**
Determining the number of boundary conditions can be challenging, especially if the boundary can be split in multiple separate components. There are two important points we should consider:
1. The minimum number of boundary conditions needed is equal to the order of the PDE (that is the order of the highest derivative in the governing equation).
2. The behaviour around ***all*** boundaries should be defined. 
3. The boundary conditions should ***not*** contradict each other (can be the case when there are *too* many boundary conditions defined, without ensuring the physical validity of the BCs).

### **Initial Conditions**
Initial conditions specify the state of the system at the beginning of the time domain. For time-dependent problems, initial conditions are necessary to start the simulation. For example, in a heat conduction problem, the initial temperature distribution must be specified.

Initial conditions are easy to define. The rule is that for a *(n-th)*-order derivative in time present in the governing equation, we need *n-1* initial conditions that are of lower order-derivative than the one present in the governing equation.

# *Physical problem : Poisson equation*

Formally, the problem to solve is: find the scalar field $u(\mathbf{x}, t)$ such that

\begin{cases} 
-\Delta u = f(\mathbf{x}) & \text{in } \Omega, \\
u = g(\mathbf{x}) & \text{on } \Gamma_D, \\
\nabla u \cdot \mathbf{n} = h(\mathbf{x}) & \text{on } \Gamma_N,
\end{cases}

where the domain and boundaries are defined as:

<div style="text-align: center;">
  <img src="../Gridap_tutorials_TUDelft/images/domain_Poisson_Eq.png" alt="Domain Poisson" width="300"/>
</div>

$$
\Omega = [0, 1] \times [0, 1].
$$

and 

$$
f(\mathbf{x}) = 1.0
$$

$$
g(\mathbf{x}) = 2.0
$$

$$
h(\mathbf{x}) = 3.0
$$

Very first step in Gridap, is to always define what we already know. In this case we know the source function $f$ and we know the values at the boundaries, which are given by the function $g$ and $h$. Moreover, we have defined the domain.

In [578]:
#  Import necessary packages

using Gridap
using Plots
using GridapMakie
using CairoMakie 
using GLMakie
using Gridap.Geometry

# Define known parameters/functions
f(x) = 1.0
g(x) = 2.0
h(x) = 3.0

# Define a square domain 
domain = (0.0, 1.0, 0.0, 1.0) # ([0,1] in x-direction and [0,1] in y-direction)

# We define the mesh resolution
partition = (10,10)  # 10 by 10 

# Finally, we create a model which is an "object" we can work with later in the tutorial.
model = CartesianDiscreteModel(domain, partition)

CartesianDiscreteModel()

Now from this model we want to find the "labels". Gridap automatically labels the entities of the model. The entities are the vertices (points), edges (lines), faces (surfaces) and cells (volumes). Gridap uses numbers, but it is more convenient to rename those entities to our liking. 

But first, let us print the initial labels of Gridap!

In [579]:
labels = get_face_labeling(model)
# 0-faces: vertices: points
# 1-faces: edges: lines
# 2-faces: faces: surfaces 
# 3-faces: cells: volumes 

FaceLabeling:
 0-faces: 121
 1-faces: 220
 2-faces: 100
 tags: 10
 entities: 9

<div style="text-align: center;">
  <img src="../Gridap_tutorials_TUDelft/images/labels_Gridap.png" alt="Gridap labels" width="300"/>
</div>

Since we are considering the Poisson equation in 2D, we find that we have a total of 9 entities:

* We have 4 vertices (4 points that connect the edges)
* We have 4 edges (4 lines that form the boundary)
* We have 1 face (1 surface that represents the domain)

All these *entities* together make 9. Now the following step is to gives names to these entities, such that later when we solve the problem we can easily assign the boundary conditions. We can check this by plotting it using *CairoMakie*, which is a Julia package for plotting. The code below is purely for visualization reasons and should NOT be studied. 

In [580]:
initial_tags = get_tag_entities(labels)[end]

# Set up the figure and axis (3D if model is 3D)
fig = Figure(resolution=(800, 600))

# In case our domain is 3D
face_colors = [
    :red,       
    :blue,      
    :green,     
    :yellow,    
    :orange,    
    :purple     
]

index_color = 1

if num_cell_dims(model) == 3
    ax = Axis3(fig[1, 1], title="Domain with Boundary Tags", xlabel="x", ylabel="y", zlabel="z")
else
    ax = Axis(fig[1, 1], title="Domain with Boundary Tags", xlabel="x", ylabel="y")
end

# Plot the domain
if num_cell_dims(model) == 3
    CairoMakie.mesh!(ax, Triangulation(model))
else
    CairoMakie.plot!(ax, Triangulation(model))
end

# Plot boundaries and vertices for each tag
for tag in initial_tags
    # Create the boundary for the current tag
    if length(initial_tags) >= 10 && tag < 10
        tag_string = "tag_0"*string(tag)
        Γ = Boundary(Triangulation(model), tags=tag_string)
    else
        tag_string = "tag_"*string(tag)
        Γ = Boundary(Triangulation(model), tags=tag_string)
    end
    try    
        # Ensure the boundary is valid and plot it
        if Γ !== nothing
            if num_cell_dims(model) == 3
                CairoMakie.mesh!(ax, Γ, transparency=false, label="Boundary " * string(tag), color=face_colors[index_color])
                index_color = index_color+1
            else
                CairoMakie.wireframe!(ax, Γ, linewidth=5, label="Boundary " * string(tag))
            end
        else
            println("Warning: Boundary for tag_", tag, " is empty or invalid.")
        end
    catch e
        # println("Error processing tag_", tag, ": ", e)
        # Γ = Boundary(Triangulation(model), tags=tag_string)
        # CairoMakie.scatter!(Γ, marker=:star8, label="vertex "*string(tag), markersize=10)
    end
end

# Add the legend to the axis
if num_cell_dims(model) < 3
    axis_legend = Legend(fig, ax)
    fig[1, 2] = axis_legend  # Place the legend in the second column
end

# # Save the plot (optional)
# save("./images/domain_with_boundaries.png", fig)

# Display the figure
fig

So from the above cel code we have the numbered labels of the model, and now we want to create new *texted* tags from the numbered labels. 

In [581]:
add_tag_from_tags!(labels, "Dirichlet", [5,7,8]) # We join all the vertices = [5,7,8] to "Dirichlet" since on these vertices we have the Dirichlet boundary condition.
add_tag_from_tags!(labels, "Neumann", [6]);  # We make vertex = 7 to "Neumann" since on this vertex we have the Neumann boundary condition.

### Triangulation and Boundary Extraction in Gridap

In $\textbf{Gridap}$, a $\texttt{Triangulation}$ is used to divide the computational domain into smaller, manageable pieces (such as triangles or tetrahedra) for numerical calculations. This forms the foundation for solving Partial Differential Equations (PDEs) using the Finite Element Method (FEM). 

The domain, denoted as $\Omega$, is represented by its triangulation:
$$
\Omega = \texttt{Triangulation(model)}
$$

which now means we express our domain and its boundaries using a set of $n$ triangles, let call this set $\{\mathcal{T}_{i}\}_{i=1}^{n}$, where all the triangles are connected together forming a "puzzle", each piece of the triangulation we can call a ***triangulation element***. This concept of triangulation is crucial for FEM, since we will later on consider each triangulation component as a sub-component of the entire problem. Hence, we solve for each "puzzle" piece, such that eventually we can place all the pieces together to solve the entire "puzzle", i.e. the physical problem.

From this triangulation, specific parts of the boundary are extracted using $\texttt{Boundary}$ with tags:

$$
\Gamma_D = \texttt{Boundary}(\Omega, \text{tags="Dirichlet"})
$$
This corresponds to the sections of the boundary where $\textbf{Dirichlet boundary conditions}$ are applied. These conditions fix the value of the solution, as discussed before.

$$
\Gamma_N = \texttt{Boundary}(\Omega, \text{tags="Neumann"})
$$
This identifies the sections of the boundary where $\textbf{Neumann boundary conditions}$ are applied. These conditions specify the fluxes, as discussed before.

By tagging these boundaries, specific physical conditions can be applied to different parts of the domain. This step is essential to define the mathematical problem that Gridap will solve. This will become more clear later on in the tutorial.


In [582]:
Ω = Triangulation(model)
Γ_D = Boundary(Ω,tags="Dirichlet")
Γ_N = Boundary(Ω,tags="Neumann")

CompositeTriangulation()

We are able to visualize this effect of triangulation, by using the CairoMakie package

In [592]:
fig = Figure(resolution=(800, 600))

ax = Axis(fig[1, 1], title="Domain with texted boundaries", xlabel="x", ylabel="y")

CairoMakie.wireframe!(ax, Ω, color=:black, linewidth=2)
CairoMakie.wireframe!(ax, Γ_D, color=:yellow, linewidth=10, label="Dirichlet")
CairoMakie.wireframe!(ax, Γ_N, color=:purple, linewidth=10, label="Neumann")
CairoMakie.scatter!(ax, Ω, marker=:star8, markersize=20, color=:green)

axis_legend = Legend(fig, ax)
fig[1, 2] = axis_legend  # Place the legend in the second column

fig

# **Finite Element spaces**

Now having completely defined the physical problem, we can go back to our paper and do some maths! Our goal is to find, in this case, the unknown dependent variable $u(\mathbf{x})$. To do so, we opt to approximate $u$ such that $u(\mathbf{x}) \approx u_{h}(\mathbf{x}) = \sum_{i}^{n} \mathbf{\tilde{u}}_{i} \phi_{i}(\mathbf{x})$. 

In other words, if we consider the *j*-th ***triangulation element***, say element $\mathcal{T}_{j}$ in our domain $\Omega$, and we consider a *pre-defined function* $\phi_{j}(\mathbf{x})$, which we call *basis function*(or *shape functions*), such that it "lives" within this element $\mathcal{T}_{j}$ and is "weighted" by $\mathbf{\tilde{u}}_{j}$. In this case, our unknown is $\mathbf{\tilde{u}}_{j}$. Now we can do the exact same thing for *all* $n$ triangulation elements and "stitch" them together such that we get ***one*** function which is our approximated solution $u_{h}(\mathbf{x}) = \sum_{i}^{n} \mathbf{\tilde{u}}_{i} \phi_{i}(\mathbf{x})$. Therefore, to find this approximated solution, often called the ***trial solution***, we need to find *all* the "weights" for each triangulation. We call these weights, the ***degrees of freedom***, which we solve for. 

However, note that we *pre-define* all the basis-functions $\phi_{i}(\mathbf{x})$, which acts as an interpolation function within the triangulation elements and between the triangulation components of the model. The *pre-defined* basis-functions are only dependent on the space $\mathbf{x}$ and are not some unknown functions we solve for. However, the choice of these basis-functions is crucial due to their corresponding characteristics and influences on the solution in the PDE. In other words, we need to find a collection of basis-functions that satisfy our problem, i.e. a ***Finite Element space***.

 In order to know what kind of basis-functions to use, we first need to derive the so-called ***weak-form*** of the problem. 

## **Weak form**
The physical problem as defined previously is called the ***strong form***. The solution of the *strong form* is, for complicated PDEs, often very difficult to find. Instead, we can reformulate the problem to the ***weak form***. To obtain the *weak form* we multiply our governing equation with a ***test function***, call it $v(\mathbf{x})$, and integrate the left and right-hand side of the governing equation along the considered domain.   

First, we start with the strong form of the Poisson equation:
$$
-\Delta u = f \quad \text{in } \Omega
$$

Next, we multiply both sides by a test function $v(\mathbf{x})$ and integrate over the domain $\Omega$:

$$
\int_{\Omega} v (-\Delta u) \, d\Omega = \int_{\Omega} v f \, d\Omega
$$

Until now, we haven't reached anything significant. Our problem still contains a second-order derivative and we now have to deal with a integral. It seems as if the problem got worse. However, the current weak form allow us to use the rule of ***integration by parts***. And this has major advantages, because of two main things:
1. Integration by parts ***naturally*** spits out a boundary component, which we can leverage to ***directly*** implement our *Neumann* boundary condition.
2. By integrating by parts, we lower the order of our PDE. This allow us to define a ***Finite Element space*** that is "well-behaved", such that when we perform integration on the basis-functions from this ***Finite Element space*** we don't run into weird computations (such as infinities).

Using integration by parts (or the divergence theorem), we can transform the left-hand side:

$$
\int_{\Omega} v (-\Delta u) \, d\Omega = \int_{\Omega} \nabla v \cdot \nabla u \, d\Omega - \int_{\partial \Omega} v (\nabla u \cdot \mathbf{n}) \, d\Gamma
$$

Here,  $\partial \Omega$ represents the boundary of the domain $\Omega$, and $\mathbf{n}$ is the outward-pointing unit normal vector on the boundary.

Substituting this back into our equation, we get the weak form of the Poisson equation:
$$
\int_{\Omega} \nabla v \cdot \nabla u \, d\Omega - \int_{\partial \Omega} v (\nabla u \cdot \mathbf{n}) \, d\Gamma = \int_{\Omega} v f \, d\Omega
$$

<!-- Applying the boundary conditions, we have:

- Dirichlet boundary condition: \( u = g \) on \( \Gamma_D \)
- Neumann boundary condition: \( \nabla u \cdot \mathbf{n} = h \) on \( \Gamma_N \)

The boundary integral term can be split into contributions from \( \Gamma_D \) and \( \Gamma_N \):

$$
\int_{\partial \Omega} v (\nabla u \cdot \mathbf{n}) \, d\Gamma = \int_{\Gamma_D} v (\nabla u \cdot \mathbf{n}) \, d\Gamma + \int_{\Gamma_N} v (\nabla u \cdot \mathbf{n}) \, d\Gamma
$$

Since \( u = g \) on \( \Gamma_D \), the term \( \int_{\Gamma_D} v (\nabla u \cdot \mathbf{n}) \, d\Gamma \) vanishes. Therefore, the weak form becomes:

$$
\int_{\Omega} \nabla v \cdot \nabla u \, d\Omega - \int_{\Gamma_N} v h \, d\Gamma = \int_{\Omega} v f \, d\Omega
$$

This is the weak form of the Poisson equation with the given boundary conditions. -->


In [577]:
order = 1
reffe = ReferenceFE(lagrangian, Float64, order)
U = TrialFESpace(Ω, reffe; conformity=:H1, dirichlet_tags="Dirichlet")
V = TestFESpace(V, g)

MethodError: MethodError: no method matching TrialFESpace(::BodyFittedTriangulation{2, 2, CartesianDiscreteModel{2, Float64, typeof(identity)}, CartesianGrid{2, Float64, typeof(identity)}, Gridap.Arrays.IdentityVector{Int64}}, ::Tuple{Lagrangian, Tuple{DataType, Int64}, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}}; conformity::Symbol, dirichlet_tags::String)

Closest candidates are:
  TrialFESpace(!Matched::Gridap.FESpaces.SingleFieldFESpace, ::Any) got unsupported keyword arguments "conformity", "dirichlet_tags"
   @ Gridap C:\Users\Kelsa\.julia\packages\Gridap\dKMpW\src\FESpaces\TrialFESpaces.jl:22


In [537]:
degree = 2
dΩ = Measure(Ω,degree)
dΓ_D  = Measure(Γ_D,degree)
dΓ_N = Measure(Γ_N,degree)

GenericMeasure()

In [538]:
a(u,v) = ∫( ∇(v)⋅∇(u) )*dΩ
b(v) = ∫( v*f )*dΩ + ∫( v*h )*dΓ_N

b (generic function with 1 method)

In [539]:
op = AffineFEOperator(a,b,U,V)

AffineFEOperator()

In [508]:
ls = LUSolver()
solver = LinearFESolver(ls)

uh = solve(solver,op)

SingleFieldFEFunction():
 num_cells: 100
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 14591716719703299956

In [509]:
writevtk(Ω,"results",cellfields=["uh"=>uh])

(["results.vtu"],)